<a href="https://colab.research.google.com/github/patrickbryant1/Umol/blob/master/Umol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Umol** - **U**niversal **mol**ecular framework
**Structure prediction of protein-ligand complexes from sequence information**

The protein is represented with a multiple sequence alignment and the ligand as a SMILES string, allowing for unconstrained flexibility in the protein-ligand interface. At a high accuracy threshold, unseen protein-ligand complexes can be predicted more accurately than for RoseTTAFold-AA, and at medium accuracy even classical docking methods that use known protein structures as input are surpassed.

For local installation, see: https://github.com/patrickbryant1/Umol

Note that the structure is not relaxed here. For relaxing the protein and resolving potential clashes - see the local installation.


In [ ]:
#@title Install dependencies
#@markdown Make sure your runtime is GPU.
#@markdown In the menu above do: Runtime --> Change runtime type --> Hardware accelerator (set to GPU)

#@markdown **Press play.**

#@markdown Simply press play on each cell below and follow the instructions.

#@markdown You will have to restart the runtime after this finishes to include the new packages.
#@markdown In the menu above do: Runtime --> Restart runtime

#@markdown Don't worry about all the errors that pip give below, these are resolved in the end.
!pip install -U jaxlib==0.3.24+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install jax==0.3.24
!pip install ml-collections==0.1.1
!pip install dm-haiku==0.0.9
!pip install pandas==1.3.5
!pip install biopython==1.81
!pip install chex==0.1.5
!pip install dm-tree==0.1.8
!pip install immutabledict==2.0.0
!pip install numpy==1.21.6
!pip install scipy==1.7.3
!pip install tensorflow==2.11.0
!pip install rdkit-pypi
!pip install py3Dmol

In [ ]:
#@title Clone the Umol github repo
import shutil
try:
  shutil.rmtree('/content/Umol', ignore_errors=True)
except:
  print('')

!git clone https://github.com/patrickbryant1/Umol.git

In [3]:
#@title #Follow all steps outlined below to run Umol
#@markdown To try the **test case** 7NB4, click the box "test_case". Then press the play button to the left.
\
#@markdown If you don't want to run the test case, **leave the box blank**.
\
#@markdown The target positions are visualised on a structure generated from [ESMFold](https://www.science.org/doi/10.1126/science.ade2574).


#@markdown #Settings
#@markdown - *ID* - name \
#@markdown - **MSA** - currently no MSA search is available directly in the browser, therefore you have to provide your own MSAs in a3m format and upload them here. \
#@markdown - Generating an MSA takes a few minutes: \
#@markdown Go to https://toolkit.tuebingen.mpg.de/tools/hhblits \
#@markdown Paste your protein sequence in the search field in fasta format --> Submit. \
#@markdown When the search is finished, go to the tab "Query MSA" and "Download Full A3M" \
#@markdown Upload the MSAs here: \
#@markdown Click the folder icon (Files) to the left and select the upload file icon. Upload your files.
#@markdown Make sure to name your MSA **"ID".a3m**

#@markdown - LIGAND - smiles string of your ligand. Make sure these are canonical (e.g. as generated by RDKit)
#@markdown - SEQUENCE - protein sequence (same as used for the MSA). **Limit=400 amino acids**.
#@markdown You can crop the structure around the target region if it is too big - Umol will handle it.
#@markdown - TARGET_POS - what positions to target in the protein sequence (the binding pocket, starting at 1)
#@markdown - NUM_RECYCLES - how many recycles to use in the network (increase for harder targets)


import sys, os
from google.colab import files
import pandas as pd
import numpy as np
import py3Dmol

sys.path.insert(0,'/content/Umol/src')
test_case = True #@param {type:"boolean"}
ID = "7NB4" #@param {type:"string"}
LIGAND = "CCc1sc2ncnc(N[C@H](Cc3ccccc3)C(=O)O)c2c1-c1cccc(Cl)c1C" # @param {type:"string"}
SEQUENCE = "SEDELYRQSLEIISRYLREQATGAKDTKPMGRSGATSRKALETLRRVGDGVQRNHETAFQGMLRKLDIKNEDDVKSLSRVMIHVFSDGVTNWGRIVTLISFGAFVAKHLKTINQESCIEPLAESITDVLVRTKRDWLVKQRGWDGFVEFFH" #@param {type:"string"}
TARGET_POSITIONS = "51,52,54,55,56,57,58,59,60,61,62,63,65,66,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,93,94,95,96,97,98,99,100,101,102,104,105,125,128,129" #@param {type:"string"}
TARGET_POSITIONS = np.array([int(x) for x in TARGET_POSITIONS.split(',')])
NUM_RECYCLES = 3 # @param {type:"integer"}
OUTDIR="/content/"+ID+'/'
if not os.path.exists(OUTDIR):
  os.mkdir(OUTDIR)

 #Write fasta
with open('/content/'+ID+'.fasta', 'w') as file:
  file.write('>'+ID+'\n')
  file.write(SEQUENCE)
FASTA_FILE='/content/'+ID+'.fasta'

#Check MSA
if test_case!=True:
  from check_msa_colab import process_a3m
  MSA='/content/'+ID+'.a3m'
  PROCESSED_MSA=MSA.split('.')[0]+'_processed.a3m'
  process_a3m(MSA, SEQUENCE, PROCESSED_MSA)
  MSA=PROCESSED_MSA
else:
  MSA='/content/Umol/data/test_case/'+ID+'/'+ID+'.a3m'


print('Using ligand:', LIGAND)
#print('Using MSA:' ,MSA)
print('Using',NUM_RECYCLES,'recycles')

#Visualise with ESMFold
if not os.path.exists(OUTDIR+'/'+ID+'_esmfold.pdb'):
  !curl -k -X POST --data $SEQUENCE  https://api.esmatlas.com/foldSequence/v1/pdb/ >> $OUTDIR/$ID'_esmfold.pdb'


view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(OUTDIR+ID+"_esmfold.pdb",'r').read(),'pdb')
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})

#Highlight
with open(OUTDIR+ID+"_esmfold.pdb","r") as ifile:
    system = "".join([x for x in ifile])
i = 0
for line in system.split("\n"):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue
    idx = int(split[5])
    if idx in TARGET_POSITIONS:
        color = "green"
        view.setStyle({'model': -1, 'serial': i+1}, {"stick": {'color': color}})
    i += 1

view.zoomTo()
view.show()

print('The target structure is in cartoon and the target positions (pocket) in stick format.')

#Subtract 1 from the target positions to make them zero indexed
TARGET_POSITIONS -= 1

Using ligand: CCc1sc2ncnc(N[C@H](Cc3ccccc3)C(=O)O)c2c1-c1cccc(Cl)c1C
Using 3 recycles


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

The target structure is in cartoon and the target positions (pocket) in stick format.


In [4]:
#@markdown #Get the parameters (if not downloaded)
if not os.path.exists('/content/params40000.npy'):
  print('Getting Umol network parameters.')
  !wget https://zenodo.org/records/10048543/files/params40000.npy
#@markdown #Make the input features for the network
from make_msa_seq_feats_colab import process
import pickle
#Make MSA feats
feature_dict = process(FASTA_FILE, [MSA])
#Write out features as a pickled dictionary.
features_output_path = os.path.join(OUTDIR, 'msa_features.pkl')
with open(features_output_path, 'wb') as f:
    pickle.dump(feature_dict, f, protocol=4)
print('Saved MSA features to',features_output_path)


#Ligand feats
from make_ligand_feats_colab import bonds_from_smiles
#Atom encoding - no hydrogens
atom_encoding = {'B':0, 'C':1, 'F':2, 'I':3, 'N':4, 'O':5, 'P':6, 'S':7,'Br':8, 'Cl':9, #Individual encoding
                'As':10, 'Co':10, 'Fe':10, 'Mg':10, 'Pt':10, 'Rh':10, 'Ru':10, 'Se':10, 'Si':10, 'Te':10, 'V':10, 'Zn':10 #Joint (rare)
                 }

#Get the atom types and bonds
atom_types, atoms, bond_types, bond_lengths, bond_mask = bonds_from_smiles(LIGAND, atom_encoding)

ligand_inp_feats = {}
ligand_inp_feats['atoms'] = atoms
ligand_inp_feats['atom_types'] = atom_types
ligand_inp_feats['bond_types'] = bond_types
ligand_inp_feats['bond_lengths'] = bond_lengths
ligand_inp_feats['bond_mask'] = bond_mask
#Write out features as a pickled dictionary.

features_output_path = os.path.join(OUTDIR, 'ligand_inp_features.pkl')
with open(features_output_path, 'wb') as f:
    pickle.dump(ligand_inp_feats, f, protocol=4)
print('Saved ligand features to',features_output_path)

Saved MSA features to /content/7NB4/msa_features.pkl
Saved ligand features to /content/7NB4/ligand_inp_features.pkl


In [ ]:
#@markdown #Predict the protein-ligand structure (a few minutes)
from net.model import config
from predict_colab import predict
import numpy as np
MSA_FEATS='/content/'+ID+'/msa_features.pkl'
LIGAND_FEATS='/content/'+ID+'/ligand_inp_features.pkl'
PARAMS=np.load('/content/params40000.npy', allow_pickle=True)

#Predict
predict(config.CONFIG,
            MSA_FEATS,
            LIGAND_FEATS,
            ID,
            TARGET_POSITIONS,
            PARAMS,
            NUM_RECYCLES,
            outdir=OUTDIR)
from relax.align_ligand_conformer_colab import read_pdb, generate_best_conformer, align_coords_transform, write_sdf
import pandas as pd
RAW_PDB=OUTDIR+'/'+ID+'_pred_raw.pdb'
#Get a conformer
pred_ligand = read_pdb(RAW_PDB)
best_conf, best_conf_pos, best_conf_err, atoms, nonH_inds, mol, best_conf_id  = generate_best_conformer(pred_ligand['chain_coords'], LIGAND)

#Align it to the prediction
aligned_conf_pos = align_coords_transform(pred_ligand['chain_coords'], best_conf_pos, nonH_inds)

#Write sdf
write_sdf(mol, best_conf, aligned_conf_pos, best_conf_id, OUTDIR+ID+'_pred_ligand.sdf')
!grep ATOM $RAW_PDB > $OUTDIR/$ID'_pred_protein.pdb'

#Ligand plDDT
!grep HETATM $RAW_PDB| cut -c65-66 > $OUTDIR/ligand_plddt.csv

In [6]:
#@markdown #Visualise
import py3Dmol
import pandas as pd
import numpy as np

ligand_plddt = pd.read_csv(OUTDIR+'ligand_plddt.csv', header=None)
print('The average ligand plDDT is:', np.round(ligand_plddt[0].mean(),1))

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(OUTDIR+ID+"_pred_ligand.sdf",'r').read(),'sdf')
view.setStyle({'stick': {'color':'cyan'}})
view.addModel(open(OUTDIR+ID+"_pred_protein.pdb",'r').read(),'pdb')
view.setStyle({'chain':'A'},{'cartoon': {'color':'green'}})
view.zoomTo()
view.show()

The average ligand plDDT is: 84.6


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Download results
from google.colab import files
files.download(OUTDIR+ID+"_pred_ligand.sdf")
files.download(OUTDIR+ID+"_pred_protein.pdb")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>